# GTHA housing market database
# OSEMN methodology Step 2: Scrub
# Step 2.3 addition of new attributes to the Teranet dataset

---

This notebook describes Step 2.3 (part of _Step 2: Scrub_ of OSEMN methodology) performed on the Teranet dataset.

Step 2.3 focuses on the addition of several new attributes to the Teranet dataset. Plan for the addition of the new attributes is presented below.

Previous steps included: 

* **Step 2.1:** spatial join between the Teranet points and the polygons of GTHA Dissemination Areas (DAs)
    
    * During step 2.1, Teranet records whose coordinates fall outside of the GTHA boundary (as defined by the DA geometry) have been filtered out (6,803,691 of the original 9,039,241 Teranet records remain in the dataset)
     
    * In addition to that, three new columns (`OBJECTID`, `DAUID`, and `CSDNAME`) derived from DA attributes have been added to each Teranet transaction

    * for details, see `notebooks/2.scrub/2.1_teranet_gtha_spatial_join.ipynb`

* **Step 2.2:** correction for consistency of the Teranet records

    * column names were converted to lower case
    
    * inconsistent capitalizations were fixed for columns
    
        * `municipality`    
        * `street_name`
        * `street_designation`
        * `postal_code` (did not show problems, converted as a preventive measure)
        
    * columns `province` and `street_suffix` were removed from the dataset
    
    * new column `street_name_raw` was created: reserve copy of unmodified `street_name`
    
    * column `street_name` was parsed and cleaned for:
    
        * `postal_code`
        * `unitno`
        * `street_number`
        * `street_direction`
        * `street_designation`
        
    * plots of the count and percentage of missing values per column were produced
    
    * inconsistent entries were fixed in the following columns:
        
        * `street_direction`
        * `street_designation`
        * `municipality`
        * `street_name`
        * `unitno`
        
    * for details, see `notebooks/2.scrub/2.2_teranet_consistency.ipynb`

---

For description of OSEMN methodology, see `methodology/0.osemn/osemn.pdf`.

For background information, description of the Teranet dataset, and its attributes, see `methodology/1.obtain/obtain.pdf`.

For description of _Step 2: Scrub_ of OSEMN methodology, see `methodology/2.scrub/scrub.pdf`.

For description of the cleanup plan for the Teranet dataset, see `methodology/2.scrub/teranet_cleanup_plan.pdf`.

For description of Step 2.1 of the cleanup process, see `notebooks/2.scrub/2.1_teranet_gtha_spatial_join.ipynb`.

For description of Step 2.2 of the cleanup process, see `notebooks/2.scrub/2.2_teranet_consistency.ipynb`.

## Plan for the addition of the new attributes

### Previously added attributes

Previously, the following new attributes were created in the Teranet dataset:

* attributes produced from the spatial join with DA geometry:

    * `objectid`: an identifier for Dissemination Areas (DAs), added as a backup identifier for DAs

    * `dauid`: another identifier for Dissemination Areas, indented to be used as the **_foreign key_** linking Teranet records with DAs (will become the **_primary key_** of DA-level datasets (_e.g.,_ DA-level Census data)
    
    * `csdname`: municipality name according to Census data (DA-level)

These attributes were added to each Teranet record via a spatial join of Teranet points with the polygons of Dissemination Areas (DAs) during Step 2.1 of the cleanup process

* attributes produced during the correction of Teranet records for consistency:

    * `street_name_raw`: unmodified reserve copy of the original `street_name` from the Teranet dataset

### Attributes to be added in this step

In this step, the following attributes will be added to the Teranet dataset:

* surrogate key:

    * `transaction_id`: unique identifier for each Teranet transaction 
    
Essentially, a simple range index, which represents the row number of a record in the full Teranet dataset (filtered to include only GTHA records), ordered by date (from earliest to latest) and `pin`
    
* attributes for display

    * `date_disp`: `registration_date` converted to `datetime.date` data type to exclude the timestamp (original `registration_date` is stored in NumPy's `datetime64` format to allow more efficient datetime operations)
    
    * `price_disp`: `consideration_amt` formatted to include thousands separator (_e.g.,_ '3,455,122') and stored as a string, for display purposes
    
* attributes for record grouping
    
    * `year`: year parsed from `registration_date`, to simplify record grouping
    
    * `3year`: `registration_date` parsed for 3-year intervals (_e.g.,_ '2014-2016'), to simplify record grouping
    
    * `5year`: `registration_date` parsed for 5-year intervals (_e.g.,_ '2012-2016'), to simplify record grouping
    
    * `10year`: `registration_date` parsed for 3-year intervals (_e.g.,_ '2007-2017'), to simplify record grouping
    
    * `xy`: `x` and `y` coordinates concatenated together (_e.g.,_ '43.098324_-79.234235'), can be used to identify and group records by their coordinate pairs
    
* correction of `consideration_amt` for inflation    
    
    * `price_infl`: `consideration_amt` corrected for inflation
    
* exploratory attributes

    * `pin/xy_total_sales`: total records for this `pin`/`xy`

    * `pin/xy_prev_sales`: previous records from this `pin`/`xy` (not counting current transaction)

    * `pin/xy_price_cum_sum`: cumulative price of all records to date from this `pin`/`xy`

    * `pin/xy_price_pct_change`: price percentage change compared to previous record from this `pin`/`xy`

    * `price_da_pct_change`: price percentage change compared to previous record from this DA (by `da_id`)

    * `pin/xy_years_since_last_sale`: years since last sale from this `pin`/`xy`

    * `da_days_since_last_sale`, `da_years_since_last_sale`: days or years since last sale from this DA (by `da_id`)

    * `sale_next_6m/1y/3y`: "looks into the future" to see whether there is another transaction from this `pin`/`xy` within the given time horizon (6 months, 1 year, 3 years)
    

## Import dependencies

In [1]:
import numpy as np
import pandas as pd
import os
from time import time

In [2]:
data_path = '../../data/teranet/'
os.listdir(data_path)

['Teranet_consistent.csv', 'Teranet_with_DA_cols.csv', 'HHSaleHistory.csv']

## Load Teranet data

In [3]:
t = time()

df = pd.read_csv(data_path + 'Teranet_consistent.csv',
                 parse_dates=['registration_date'], low_memory=False)
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:,.2f} seconds ({1:.2f} minutes)".format(elapsed, elapsed / 60) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

----- DataFrame loaded
in 21.15 seconds (0.35 minutes)
with 6,803,691 rows
and 17 columns
-- Column names:
 Index(['lro_num', 'pin', 'consideration_amt', 'registration_date',
       'postal_code', 'unitno', 'street_name', 'street_designation',
       'street_direction', 'municipality', 'street_number', 'x', 'y',
       'objectid', 'dauid', 'csdname', 'street_name_raw'],
      dtype='object')


In [4]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6803691 entries, 0 to 6803690
Data columns (total 17 columns):
lro_num               6803691 non-null int64
pin                   6803691 non-null int64
consideration_amt     6803691 non-null float64
registration_date     6803691 non-null datetime64[ns]
postal_code           6233365 non-null object
unitno                1572959 non-null object
street_name           6598317 non-null object
street_designation    6522418 non-null object
street_direction      683462 non-null object
municipality          6799681 non-null object
street_number         6594324 non-null object
x                     6803691 non-null float64
y                     6803691 non-null float64
objectid              6803691 non-null int64
dauid                 6803691 non-null int64
csdname               6803691 non-null object
street_name_raw       6598317 non-null object
dtypes: datetime64[ns](1), float64(3), int64(4), object(9)
memory usage: 882.4+ MB


## Surrogate key

### Add attribute `transaction_id`
Attribute `transaction_id` is intended as a unique identifier for each record in the Teranet dataset. It will be used as the **_primary key_** for records in Teranet table in the proposed GTHA housing market database. It is produced as a surrogate key, as no other attribute or combination of attributes in the Teranet dataset allows the records to be uniquely identified. 
    
`transaction_id` is essentially a simple range index, which represents the row number of a record in the full Teranet dataset (filtered to include only GTHA records), ordered by date (from earliest to latest) and `pin`

#### Order Teranet records by `registration_date` and `pin`

In [5]:
df = df.sort_values(['registration_date', 'pin'])
print("DataFrame was resorted by 'registration_date' and 'pin'.")

DataFrame was resorted by 'registration_date' and 'pin'.


#### Insert the new column `transaction_id`

In [6]:
df.insert(0, "transaction_id", np.arange(len(df)))
print("New column 'transaction_id' was added to the DataFrame.")

New column 'transaction_id' was added to the DataFrame.


## Attributes for display

### Add attribute `date_disp`
Since Teranet records do not carry any actual timestamp (each record has a timestamp of '00:00:00'), a new column `date_disp` is created from `registration_date` to show only dates. The new column `date_disp` has a data type of `datetime.date`, while the original column `registration_date` is stored in NumPy's `datetime64` data type, which allows more efficient datetime operations.

In [7]:
df['date_disp'] = df['registration_date'].dt.date
print("New column 'date_disp' was added to the DataFrame.")

New column 'date_disp' was added to the DataFrame.


### Add attribute `price_disp`
A new column `price_disp` is created from values of `consideration_amt` formatted to include thousands separator (_e.g.,_ '3,455,122') and stored as a string, for display purposes.

In [8]:
df['price_disp'] = df['consideration_amt'].apply(lambda x: '{:,}'.format(x))
print("New column 'price_disp' was added to the DataFrame.")

New column 'price_disp' was added to the DataFrame.


## Add attribute `year`
The new attribute `year` is parsed from `registration_date` and stored as a string, to simplify record grouping.

In [9]:
df['year'] = df['date_disp'].astype('str').apply(lambda x: x[:4])
print("New column 'year' was added to the DataFrame.")

New column 'year' was added to the DataFrame.


## Add attribute `3year`
The new attribute `3year` is created from the column `year` parsed for 3-year intervals (_e.g.,_ '2014-2016') and stored as a string, to simplify record grouping.

In [10]:
t = time()

df['3year'] = df['year']

year_list = df['year'].unique()

i = 0
ylist = []
for year in year_list:
    ylist.append(year)
    i += 1
    if i == 3:
        df['3year'] = df['3year'].str.replace('^' + ylist[0] + '$|^' + ylist[1] + '$|^' + ylist[2] + '$', 
                                              ylist[0] + '-' + ylist[2])
        ylist = []
        i = 0

elapsed = time() - t

print("New column '3year' was added to the DataFrame. Took {0:,.2f} seconds ({1:.2f} minutes)."
      .format(elapsed, elapsed / 60))
df['3year'].value_counts().sort_index()

New column '3year' was added to the DataFrame. Took 326.91 seconds (5.45 minutes).


1805-1811         5
1815-1817         6
1819-1821         7
1822-1824         9
1827-1829        13
              ...  
2005-2007    798539
2008-2010    729790
2011-2013    760386
2014-2016    862676
2017         218843
Name: 3year, Length: 68, dtype: int64

## Add attribute `5year`
The new attribute `5year` is created from the column `year` parsed for 5-year intervals (_e.g.,_ '2012-2016') and stored as a string, to simplify record grouping.

In [11]:
t = time()

df['5year'] = df['year']

year_list = df['year'].unique()[1:] # skipping first year (1805) to match the length of a 5-year window

i = 0
ylist = []
for year in year_list:
    ylist.append(year)
    i += 1
    if i == 5:
        df['5year'] = df['5year'].str.replace('^' + ylist[0] + '$|^' + ylist[1] + '$|^' + ylist[2] + '$|^' +
                                              ylist[3] + '$|^' + ylist[4] + '$', 
                                              ylist[0] + '-' + ylist[4])
        ylist = []
        i = 0

elapsed = time() - t

print("New column '5year' was added to the DataFrame. Took {0:,.2f} seconds ({1:.2f} minutes)."
      .format(elapsed, elapsed / 60))
df['5year'].value_counts().sort_index()

New column '5year' was added to the DataFrame. Took 195.34 seconds (3.26 minutes).


1805               1
1809-1817         10
1819-1823         11
1824-1831         23
1832-1836         14
1837-1841         17
1842-1846         39
1847-1851        119
1852-1856        743
1857-1861        203
1862-1866        118
1867-1871        284
1872-1876        421
1877-1881        392
1882-1886        523
1887-1891        546
1892-1896        475
1897-1901        311
1902-1906       1100
1907-1911       1087
1912-1916       1845
1917-1921       1576
1922-1926       1909
1927-1931       2105
1932-1936       1152
1937-1941       1737
1942-1946       2894
1947-1951       9273
1952-1956      23820
1957-1961      36874
1962-1966      54104
1967-1971      67278
1972-1976      91223
1977-1981     128786
1982-1986     214310
1987-1991     450498
1992-1996     605472
1997-2001     970776
2002-2006    1283035
2007-2011    1267902
2012-2016    1361842
2017          218843
Name: 5year, dtype: int64

## Add attribute `10year`
The new attribute `10year` is created from the column `year` parsed for 5-year intervals (_e.g.,_ '2014-2017') and stored as a string, to simplify record grouping.

In [12]:
t = time()

df['10year'] = df['year']

year_list = df['year'].unique()[1:] # skipping first year (1805) to match the length of a 10-year window

i = 0
ylist = []
for year in year_list:
    ylist.append(year)
    i += 1
    if i == 10:
        df['10year'] = df['10year'].str.replace('^' + ylist[0] + '$|^' + ylist[1] + '$|^' + ylist[2] + '$|^' +
                                                ylist[3] + '$|^' + ylist[4] + '$|^' + ylist[5] + '$|^' + ylist[6] +
                                                '$|^' + ylist[7] + '$|^' + ylist[8] + '$|^' + ylist[9] + '$', 
                                                ylist[0] + '-' + ylist[9])
        ylist = []
        i = 0

elapsed = time() - t

print("New column '10year' was added to the DataFrame. Took {0:,.2f} seconds ({1:.2f} minutes)."
      .format(elapsed, elapsed / 60))
df['10year'].value_counts().sort_index()

New column '10year' was added to the DataFrame. Took 97.64 seconds (1.63 minutes).


1805               1
1809-1823         21
1824-1836         37
1837-1846         56
1847-1856        862
1857-1866        321
1867-1876        705
1877-1886        915
1887-1896       1021
1897-1906       1411
1907-1916       2932
1917-1926       3485
1927-1936       3257
1937-1946       4631
1947-1956      33093
1957-1966      90978
1967-1976     158501
1977-1986     343096
1987-1996    1055970
1997-2006    2253811
2007-2016    2629744
2017          218843
Name: 10year, dtype: int64

### Add attribute `xy`

The new attribute `xy` is produced by concatenating `x` and `y` together (_e.g.,_ '-79.9774202446447_43.203290987723'), it can be used to identify and group records by their coordinate pairs.

In [13]:
df['xy'] = df['x'].astype('str') + '_' + df['y'].astype('str')
print("New column 'xy' was added to the DataFrame.")

New column 'xy' was added to the DataFrame.


## Correction of `consideration_amt` for inflation

## Exploratory attributes

### Add column `total_sales`
Total records for each pin, generated as a separate DataFrame `df_pin` which represents Teranet records grouped and indexed by `pin`.

`total_sales_pin` is added as a new column for Teranet records via a merge operation on `pin`.

In [14]:
# group records by `pin`
t = time()
pin_counts = \
    df.groupby('pin')['consideration_amt'].count()
pin_counts.name = 'pin_total_sales'
df = pd.merge(df, pin_counts, on='pin')
elapsed = time() - t
print("\nNew column 'pin_total_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group records by `xy` coordinate pairs
t = time()
xy_counts = \
    df.groupby('xy')['consideration_amt'].count()
xy_counts.name = 'xy_total_sales'
df = pd.merge(df, xy_counts, on='xy')
elapsed = time() - t
print("\nNew column 'xy_total_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))


New column 'pin_total_sales' added to the DataFrame!
took 24.94 seconds.

New column 'xy_total_sales' added to the DataFrame!
took 20.06 seconds.


### Add column `prev_sales`
New columns are added to Teranet records capturing, for each transaction, a rolling count of previous records from this `pin` or `xy` coordinate pair.

In [15]:
df['count'] = 1 # used to produce rolling counts per `pin` and `xy`

# group by `pin`
t = time()
df['pin_prev_sales'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['count'].cumsum() - 1
elapsed = time() - t
print("\nNew column 'pin_prev_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group by xy pairs
t = time()
df['xy_prev_sales'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['count'].cumsum() - 1
elapsed = time() - t
print("\nNew column 'xy_prev_sales' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

df = df.drop('count', axis=1)


New column 'pin_prev_sales' added to the DataFrame!
took 13.20 seconds.

New column 'xy_prev_sales' added to the DataFrame!
took 20.61 seconds.


### Add columns `price_cum_sum` and `price_pct_change`
New columns are added to Teranet records capturing, for each transaction, a rolling sum of price from previous records from this `pin` or `xy` coordinate pair, and `pct_change` compared to previous transaction from this `pin` or `xy` pair.

In [16]:
# `price_cum_sum`
# group records by `pin`
t = time()
df['pin_price_cum_sum'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['consideration_amt'].cumsum()
elapsed = time() - t
print("\nNew column 'pin_price_cum_sum' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# group records by `xy` pairs
t = time()
df['xy_price_cum_sum'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['consideration_amt'].cumsum()
elapsed = time() - t
print("\nNew column 'xy_price_cum_sum' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# `price_pct_change`
# group records by `pin`
t = time()
df['pin_price_pct_change'] = \
    df.sort_values(['pin', 'registration_date'])\
    .groupby('pin')['consideration_amt'].pct_change()
elapsed = time() - t
print("\nNew column 'pin_price_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# group records by `xy`
t = time()
df['xy_price_pct_change'] = \
    df.sort_values(['xy', 'registration_date'])\
    .groupby('xy')['consideration_amt'].pct_change()
elapsed = time() - t
print("\nNew column 'xy_price_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))


New column 'pin_price_cum_sum' added to the DataFrame!
took 9.30 seconds.

New column 'xy_price_cum_sum' added to the DataFrame!
took 20.30 seconds.

New column 'pin_price_pct_change' added to the DataFrame!
took 10.30 seconds.

New column 'xy_price_pct_change' added to the DataFrame!
took 21.32 seconds.


## Add column `price_da_pct_change`
New column is added to Teranet records capturing, for each transaction, percentage change in price compared to the previous record from this `da_id`.

In [17]:
# add column 'price_da_pct_change' to Teranet records DataFrame 
t = time()
df['price_da_pct_change'] = \
    df.sort_values(['dauid', 'registration_date'])\
    .groupby('dauid')['consideration_amt'].pct_change()
elapsed = time() - t
print("\nNew column 'price_da_pct_change' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))


New column 'price_da_pct_change' added to the DataFrame!
took 8.90 seconds.


## Add columns `years_since_last_sale`
New columns are added to Teranet records capturing, for each transaction, years passed since the previous record from this `pin` or `xy` coordinate pair.

In [18]:
t = time()
df['pin_years_since_last_sale'] = \
    df.sort_values(['pin', 'registration_date'])\
      .groupby('pin')['registration_date']\
        .diff().dt.days / 365
elapsed = time() - t
print("New column 'pin_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

KeyboardInterrupt: 

In [ ]:
print("Adding the new column...")
# add column 'prev_visits' to Teranet records DataFrame 
t = time.time()
df['xy_years_since_last_sale'] = \
    df.sort_values(['xy', 'registration_date'])\
      .groupby('xy')['registration_date']\
        .diff().dt.days / 365
elapsed = time.time() - t
print("New column 'xy_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

## Add columns `da_days_since_last_sale` and `da_years_since_last_sale`
New columns are added to Teranet records capturing, for each transaction, years passed since the previous record from this `pin` or `xy` coordinate pair.

In [ ]:
print("Adding new columns...")
# add column 'da_days_since_last_sale' to Teranet records DataFrame 
t = time.time()
df['da_days_since_last_sale'] = \
    df.sort_values(['da_id', 'registration_date'])\
      .groupby('da_id')['registration_date']\
        .diff().dt.days
elapsed = time.time() - t
print("New column 'da_days_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# add column 'da_years_since_last_sale' to Teranet records DataFrame 
t = time.time()
df['da_years_since_last_sale'] = \
    df.sort_values(['da_id', 'registration_date'])\
      .groupby('da_id')['registration_date']\
        .diff().dt.days / 365
elapsed = time.time() - t
print("New column 'da_years_since_last_sale' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

## Add columns `sale_next_6m/1y/3y` per `pin` and `xy`
New columns are added to Teranet records capturing, for each transaction, whether there would be another transaction in the future from this `pin`, `xy`, or `da_id`

Time horizons used: 6 months, 1 year, 3 years.

In [ ]:
print("Adding new columns...")

# create a new column, marks True if next 'day_diff' <= 5

# group records by `pin`
t = time.time()
df = df.sort_values(['pin', 'registration_date'])
df['pin_sale_next_6m'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 0.5
df['pin_sale_next_1y'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 1
df['pin_sale_next_3y'] = \
    df['pin_years_since_last_sale'].shift(-1) <= 3
elapsed = time.time() - t
print("New columns 'pin_sale_next_..' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

# group records by `xy`
t = time.time()
df = df.sort_values(['xy', 'registration_date'])
df['xy_sale_next_6m'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 0.5
df['xy_sale_next_1y'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 1
df['xy_sale_next_3y'] = \
    df['xy_years_since_last_sale'].shift(-1) <= 3
elapsed = time.time() - t
print("New columns 'xy_sale_next_..' "
      "added to the DataFrame!"
      "\ntook {0:.2f} seconds.".format(elapsed))

## Save results to a new .csv file
Teranet dataset without NaN records and with 12 new columns is saved as:
`data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv`

In [ ]:
t = time.time()
save_path = 'data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv'
df.to_csv(save_path, index=False)
elapsed = time.time() - t
print("File saved to path:\n'" + save_path + "'" + 
      "\ntook {0:.2f} seconds.".format(elapsed))